In [2]:
# print version / should be 1.6.x
println(versioninfo())

# load centralized paths dictionary
using YAML
PATHS_FILE = "../../../PATHS.YML"
PATHS = YAML.load_file(PATHS_FILE);

"""
    load_path(keys::Vector{String}):String

Return the absolute path for a requested item within the nested PATHS dictionary.
"""
function load_path(keys::Vector{String}):String
    # recusively assemble paths from keys
    requested_path = foldl((x, y) -> getindex(x, y), keys; init=PATHS)
    # rephrase abs path from relative paths to be platform independent.
    realpath(joinpath(splitdir(realpath(PATHS_FILE))[1], requested_path))
end;

Julia Version 1.6.6
Commit b8708f954a (2022-03-28 07:17 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin18.7.0)
  CPU: Intel(R) Core(TM) i5-4260U CPU @ 1.40GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-11.0.1 (ORCJIT, haswell)
nothing


In [3]:
# we load the project path as it is used by CliMA Land v01 papers
using Pkg
PROJECT_PATH = load_path(["EXTERNAL", "01_CLIMA_LAND_V01", "ROOT"])
Pkg.activate(PROJECT_PATH)


  Activating environment at `~/Desktop/island_in_the_sun/python/2022_04_lsm/ext_src/01_julia_land_v01/Land/Project.toml`


# First exploration of CliMA Land v0.1

For our emulator, we want to use the CliMA Land model (v0.1), as it has [some advantages](../../../docs/reporting/01_meetings_documentation.md#M05_climaland). Therefore, it is important to understand the model. Thus, we will install it and follow/disentangle the code of the trial. Later, we can change some parameters and see the response.

## Installation

I installed CliMA Land v0.1 following the instructions of its [README](https://github.com/CliMA/Land/blob/41f90baa90c5b8e761fdcce5005eebde2f42b26b/README.md).

The script `clima-land.jl` was stored in the cloned repository, the dataset`era5_2019_117_296_1X.csv` in our cache directory.

## Disentangling the wrapper

The wrapper of the tutorial should allow to run the LSM simulations for single points. To reproduce and alternate it, I will go through the code subsequently.

**This is the whole script:**

In [7]:
# load wrapper path and print its code
CL_WRAPPER_FILE = load_path(["EXTERNAL", "01_CLIMA_LAND_V01", "WRAPPER"]);

open(CL_WRAPPER_FILE) do file
    println(read(file, String))
end

using LazyArtifacts

using CanopyLayers: BLUE, EVI, FourBandsFittingHybrid, NDVI, NIR, NIRv, NIRvES, RED, SIF_740, SIF_WL, SIF_fluxes!, canopy_fluxes!, canopy_geometry!, canopy_matrices!, fit_soil_mat!, short_wave!, soil_albedos
using DataFrames: DataFrame
using GriddingMachine: Indexer.lat_ind, Indexer.lon_ind
using Photosynthesis: AbstractPhotoModelParaSet, AirLayer, leaf_ETR!
using PkgUtility: numerical∫, read_csv, save_csv!
using PlantHydraulics: soil_p_25_swc, temperature_effects!
using SoilPlantAirContinuum: SPACMono, initialize_spac_canopy!, update_Cab!, update_LAI!, zenith_angle
using Statistics: mean
using StomataModels: BetaGLinearPsoil, CanopyLayer, ESMMedlyn, EmpiricalStomatalModel, GswDrive, gas_exchange!, gsw_control!, stomatal_conductance, update_leaf_TP!, β_factor
using UnPack: @unpack
using WaterPhysics: saturation_vapor_pressure


# function to create a SPAC node
create_spac(lat::FT, lon::FT) where {FT<:AbstractFloat} = (
    _soil_bnds = FT[0, -0.1, -0.35, -1, -3];
 

### Function per function

I will go through the functions one after the other:

#### `create_spac`

The CliMA Land model uses instances of soil plant air continua (a.k.a SPACs). The [SPACMono](https://yujie-w.github.io/SoilPlantAirContinuum.jl/stable/API/#SoilPlantAirContinuum.SPACMono) is the atomic 1D abstraction of a grid cell. It represents a single species of plant interacting with soil and atmosphere and has properties like:
- Root depth
- Canopy maximal height
- Longitude and Latitude
- Ground area
- Layers of Roots/Canopy
- Canopy water flux per ground area
- ...

The variable `soil_bnds` represents the discrete boundaries of soil layers for different hydraulic parameters. It is used as given by [van Genuchten (1980)](#vangenuchtenClosedformEquationPredicting1980).

---
# References
[**van Genuchten MTh. 1980.**](https://doi.org/10.2136/sssaj1980.03615995004400050002x) <a id="vangenuchtenClosedformEquationPredicting1980"></a> A Closed-form Equation for Predicting the Hydraulic Conductivity of Unsaturated Soils. Soil Science Society of America Journal 44: 892–898.
